Никаких ограничений в задании не было, поэтому сразу импортируем все необходимые нам библиотеки для решения:
Pandas - для работы с датафреймом;
Scipy - для некоторых расчетов в 4 задании.

In [31]:
import pandas as pd
from scipy import stats
import math

Обработаем наш файл transactions.txt согласно его пути на машине (с учетом того, что разделителем является запятая).
Так как заголовка в исходном файле по условию нет, то создадим еще и названия колонок для простоты.

In [42]:
data = pd.read_csv ('/usr/local/data/transactions.txt', sep=",", header=None)
data.columns = ['number', 'id', 'transaction', 'segment']

FileNotFoundError: [Errno 2] File b'/usr/local/data/transactions.txt' does not exist: b'/usr/local/data/transactions.txt'

Проверим, что всё корректно и выведем несколько строк датафрейма.

In [33]:
data.head (6)

,number,id,transaction,segment
0,1,ADF346274,1000,R
1,2,JDY272800,600,AF
2,3,ADF346274,2460,R
3,4,TDG254776,4000,R
4,5,RYE846234,70000,AF
5,6,ADF346274,1000,R


# Задание 1

Перейдем непосредственно к выполнению заданий. В первом задании нас просят подсчитать количество клиентов в каждом из сегментов (насколько я понимаю, тут надо подсчитать количество уникальных клиентов, так как один и тот же клиент мог сделать много транзакций за этот день). С помощью группировки по сегменту и метода nunique из pandas получаем следующее (сгруппировали по сегменту и посчитали количество уникальных id):

In [34]:
pd.DataFrame (data.groupby ('segment').nunique()['id']) #колонка 'id' и есть ответ на первый вопрос

,id
segment,
AF,2
R,2


# Задание 2

В этом задании надо посчитать среднее значение транзакции в каждом сегменте. Тоже довольно легко сделать или с помощью метода describe или aggregate (опять сгруппировали по сегменту и посчитали среднее в этом сегменте):

In [35]:
data.groupby('segment').aggregate({'transaction' : 'mean'}) #колонка 'transaction' и есть ответ на второй вопрос

,transaction
segment,
AF,35300.000000
R,2486.666667


# Задание 3

Для подсчета 90% доверительного интервала для среднего в каждом сегменте воспользуемся нормальным распределением. Общая формула доверительного интервала следующая $\left[ \overline{X}-cv\times SEM;\overline{X}+cv\times SEM \right]$, где 
$\overline{X}$ - выборочное среднее, $cv$ - критическое значение (в нашем случае это 1.645, его надо искать по таблицам нормального распределения), $SEM$ - стандартная ошибка среднего. Получить все эти данные можно с помощью describe. Остается только посчитать интервалы для каждого сегмента.

In [36]:
df = data.groupby('segment').describe()['transaction']
df

,count,mean,std,min,25%,50%,75%,max
segment,,,,,,,,
AF,12.0,35300.000000,36242.96797,600.0,600.0,35300.0,70000.0,70000.0
R,18.0,2486.666667,1260.40143,1000.0,1000.0,2460.0,4000.0,4000.0


In [37]:
#запишем всё по переменным для удобства чтения кода

# первый сегмент
mean_af = df['mean'][0]
std_af = df['std'][0]
number_af = df['count'][0]
sem_af = std_af/math.sqrt(number_af)

# второй сегмент
mean_r = df['mean'][1]
std_r = df['std'][1]
number_r = df['count'][1]
sem_r = std_r/math.sqrt(number_r)

In [38]:
print ('[',mean_af-1.645*sem_af,';',mean_af+1.645*sem_af,']') # интервал для AF сегмента

[ 18089.28018448438 ; 52510.71981551562 ]


In [39]:
print ('[',mean_r-1.645*sem_r,';',mean_r+1.645*sem_r,']') # интервал для R сегмента

[ 1997.9709450095058 ; 2975.3623883238274 ]


# Задание 4

Для проверки гипотезы о сравнении двух средних воспользуемся t-критерием Стьюдента. Гипотеза H0: среднее в AF = среднему в R, а гипотеза H1: средние в группах не равны. Для проверки гипотезы на 10% уровне значимости нам надо будет вычислить t-значение наших двух групп: $$t=\frac{\overline{{{X}_{1}}}-\overline{{{X}_{2}}}}{\sqrt{\frac{sd_{1}^{2}}{{{n}_{1}}}+\frac{sd_{2}^{2}}{{{n}_{2}}}}}$$ где $\overline{{{X}_{1}}}$ и $\overline{{{X}_{2}}}$ - средние в группах, $sd_{1}$ и $sd_{2}$ - стандартные отклонения в группах, ${n}_{1}$ и ${n}_{2}$ - размеры групп (то есть количество элементов в каждом сегменте). Потом вычисляем число степеней свободы $df={{n}_{1}}+{{n}_{2}}-2$ и находим p-value для этих параметров. Сравниваем его с 0.1 и делаем выводы в пользу той или иной гипотезы.

In [40]:
# все данные: среднее в сегментах, стандартное отклонение и количество наблюдений в группах у нас уже есть выше
t = (mean_af-mean_r)/math.sqrt((std_af*std_af)/number_af + (std_r*std_r)/number_r)
DF = number_af + number_r - 2
pval = stats.t.sf(abs(t), DF)*2 # двусторонний t-тест
print ('t-statistic = %6.3f pvalue = %6.4f' % (t, pval))

t-statistic =  3.135 pvalue = 0.0040


t-статистику и p-value посчитали выше и осталось сравнить с нашим уровнем значимости и принять или отвергнуть наши гипотезы.

In [41]:
if (pval < 0.1):
    print ('H0 rejected') # в этом случае H1 верна и есть значимые различия в средних
else:
    print ('H0 cannot be rejected') # нельзя отвергунть нулевую гипотезу

H0 rejected


P.S. Можно было бы вместо t-критерия, который не очень устойчив по отношению к выбросам использовать критерий Мана-Уитни. Однако, критерий Мана-Уитни используется обычно для не слишком больших выборок, а у нас в 250гб файле их очень много и это заняло бы огромное время для подсчета. Поэтому лучше всего использовать t-критерий и в идеале предобработать файл на выбросы.